In [84]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta, timezone
import pytz

In [63]:
# for idx in kl_fileCode.index:
#     kl_fileCode["District"][idx] = kl_fileCode["District"][idx].capitalize()

In [64]:
# kl_fileCode.to_csv("kl_districtCode.csv",index=False)

In [ ]:
class NoDataFound(Exception):
    pass

In [92]:
def GenerateRawCsv(StateCode,Date,df_districts,df_summary):
    # print(df_summary['Confirmed'].values)
    utc_dt = datetime.now(timezone.utc)
    df = pd.DataFrame(columns=["Date","State/UTCode","District","tested_last_updated_district","tested_source_district","notesForDistrict",
    "cumulativeConfirmedNumberForDistrict","cumulativeDeceasedNumberForDistrict","cumulativeRecoveredNumberForDistrict",
    "cumulativeTestedNumberForDistrict","last_updated","tested_last_updated_state","tested_source_state","notesForState",
    "cumulativeConfirmedNumberForState","cumulativeDeceasedNumberForState","cumulativeRecoveredNumberForState","cumulativeTestedNumberForState"])

    df['District'] = df_districts['District']
    if "Confirmed" in df_districts.columns:
        df['cumulativeConfirmedNumberForDistrict'] = df_districts['Confirmed']
        df['cumulativeConfirmedNumberForState'] = df_summary['Confirmed']#.astype(int).sum()
        # print(df['cumulativeConfirmedNumberForState'])
    if "Tested" in df_summary.index:
        df['cumulativeTestedNumberForState'] = df_summary['Tested'] #.astype(int).sum()
    if "Tested" in df_districts.columns:
        df['cumulativeTestedNumberForDistrict'] = df_districts['Tested']
    df['cumulativeDeceasedNumberForDistrict'] = df_districts['Deceased']
    df['cumulativeRecoveredNumberForDistrict'] = df_districts['Recovered']
    # df['cumulativeOtherNumberForDistrict'] = df_districts['Other']
    # print(Date)
    # a=b
    df['Date'] = Date
    df['State/UTCode'] = StateCode
     
    df['cumulativeRecoveredNumberForState'] = df_summary['Recovered'] #.astype(int).sum()
    df['cumulativeDeceasedNumberForState'] = df_summary['Deceased'] #.astype(int).sum()
    
    try:
        df['cumulativeOtherNumberForDistrict'] = df_districts['Other']
        df['notesForDistrict'] = df_districts['notesForDistrict']
    except:
        df['cumulativeOtherNumberForDistrict'] = 0
    try:
        df['cumulativeOtherNumberForState'] = df_summary['Other']
        df['notesForState'] = df_summary['notesForState']
    except:
        df['cumulativeOtherNumberForState'] = 0
    
    IST = pytz.timezone('Asia/Kolkata')
    df['last_updated'] = utc_dt.astimezone(IST).isoformat()
    # print(df.head(1))
    # a=b
    df.to_csv("{}_raw.csv".format(StateCode))

In [112]:
def getKLData(file_path,date,StateCode):
    kl_fileCode = pd.read_csv("kl_districtCode.csv")
    prevdate = str((datetime.strptime(date,"%Y-%m-%d")- timedelta(days=1)).date())
    Date = datetime.strptime(date,"%Y-%m-%d")
    Date = Date.strftime("%d-%m-%Y")

    df_districts = pd.DataFrame(columns=['Date', 'District', 'Confirmed', 'Recovered', 'Active', 'Death'])
    for idx in kl_fileCode.index:
        DistrictData = pd.read_html('../INPUT/KL/kerala-'+ str(kl_fileCode["Code"][idx]) +'.html')
        DistrictData = DistrictData[1]
        DistrictData["District"] = kl_fileCode["District"][idx]
        df_districts = df_districts.append(DistrictData[DistrictData["Date"] == Date])
    print(df_districts)
    
    testingData = pd.read_html('../INPUT/KL/kerala-testing.html')
    testingData = testingData[1]
    testdone = testingData[testingData["Date"] == Date]["Tot. Sent"].item()

    df_districts = df_districts.rename(columns={'Death':'Deceased'})
    df_districts.loc[len(df_districts.index)] = [Date,'Total', 0, 0, 0, 0] 

    
    # print('prevdate',prevdate ,type(prevdate))
    updated_data_frame = df_districts
    df_prevDay = pd.read_csv("../../RAWCSV/{}/{}_raw.csv".format(prevdate,StateCode))
    # df_prevDay = pd.read_csv('../RAWCSV/{}/{}_raw.csv'.format(prevdate,StateCode))

    # print('base csv is', df_prevDay)

    for index, row in df_prevDay.iterrows():
        District_base_col = row['District']
        # print('District_base_col',District_base_col)

        if District_base_col != "Total" :
            # print(District_base_col)
            filtered_dataframe= df_districts[df_districts['District'] == District_base_col]
            district_index = filtered_dataframe.index[0]
            if not filtered_dataframe.empty:

                # pdf data columns
                Confirmed_col= filtered_dataframe['Confirmed'].iloc[0]
                Recovered_col =filtered_dataframe['Recovered'].iloc[0]
                Deaths_col = filtered_dataframe['Deceased'].iloc[0]
                # print(type(Confirmed_col))
                # print('Confirmed_col',Confirmed_col, 'Recovered_col',Recovered_col,'Deaths_col',Deaths_col)

                # base data columns
                Confirmed_base_col = row['cumulativeConfirmedNumberForDistrict']
                Recovered_base_col = row['cumulativeRecoveredNumberForDistrict']
                Deaths_base_col = row['cumulativeDeceasedNumberForDistrict']
                # print('Confirmed_base_col',Confirmed_base_col,'Recovered_base_col',Recovered_base_col,'Deaths_base_col',Deaths_base_col)

                # addition

                Confirmed_col =  Confirmed_col + Confirmed_base_col
                Recovered_col = Recovered_col + Recovered_base_col
                Deaths_col = Deaths_col + Deaths_base_col
                # print('Confirmed_col',Confirmed_col,
                # 'Recovered_col',Recovered_col,
                # 'Deaths_col',Deaths_col)

                # updating dataframe column values with addition values
                updated_data_frame.at[district_index, 'Confirmed'] = Confirmed_col
                updated_data_frame.at[district_index, 'Recovered'] = Recovered_col
                updated_data_frame.at[district_index, 'Deceased'] = Deaths_col
    # print('updated_data_frame',updated_data_frame)


    # get the index of total row and set those row values to zero then update with sum values
    # finding index of total 
    index_of_total = updated_data_frame[updated_data_frame['District'] == 'Total'].index[0]

    # set the total row values to zero

    updated_data_frame.at[index_of_total, 'Confirmed'] = 0
    updated_data_frame.at[index_of_total, 'Recovered'] = 0
    updated_data_frame.at[index_of_total, 'Deceased'] = 0


    # summing of all the Confirmed, Recovered, Deceased and Other Column values 
    Confirmed_Sumvalue = updated_data_frame['Confirmed'].sum()
    Recovered_Sumvalue = updated_data_frame['Recovered'].sum()
    Deceased_Sumvalue = updated_data_frame['Deceased'].sum()


    # updating total row with sum of all the (C, R, D, O) column values
    updated_data_frame.at[index_of_total, 'Confirmed'] = Confirmed_Sumvalue
    updated_data_frame.at[index_of_total, 'Recovered'] = Recovered_Sumvalue
    updated_data_frame.at[index_of_total, 'Deceased'] = Deceased_Sumvalue

    # print('updated data frame is', updated_data_frame)

    df_districts = updated_data_frame
    df_summary = df_districts
    df_summary = df_summary.iloc[-1,:]
    df_summary["Tested"] = testdone
    df_districts = df_districts[:-1]
    df_json = pd.read_json("../../DistrictMappingMaster.json")
    dist_map = df_json['Kerala'].to_dict()
    df_districts['District'].replace(dist_map,inplace=True)
    df_summary["Date"] = date
    df_districts["Date"] = date
    return df_summary,df_districts

In [113]:
Date = "2022-05-11"
StateCode = "KL"
file_path = "../INPUT/{0}/{1}.pdf".format(Date,StateCode)
df_summary,df_districts = getKLData(file_path,Date,StateCode)
GenerateRawCsv(StateCode,Date,df_districts,df_summary)

11-05-2022
Empty DataFrame
Columns: [Date, District, Confirmed, Recovered, Active, Death]
Index: []


ValueError: can only convert an array of size 1 to a Python scalar